# Solving Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [331]:
# Install the OpenAI Python package.
%pip install openai
%pip install langchain openai chromadb gspread oauth2client pandas
%pip install --upgrade langchain openai chromadb
%pip install langchain[all]


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: openai
    Found existing installation: openai 1.50.2
    Uninstalling openai-1.50.2:
      Successfully uninstalled openai-1.50.2
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [332]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000001iX7PMAU'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hi there! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [333]:
# IMPORTS
from IPython.display import display
from typing import Union
from enum import Enum

import pandas as pd
import time as clock
import re
import random

In [334]:
# Getting data

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(export_link)

DOC_ID = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(DOC_ID, 'products')
emails_df = read_data_frame(DOC_ID, 'emails')

# Display first 3 rows of each DataFrame
display(products_df)
display(emails_df.head(10))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00
3,CSH1098,Cozy Shawl,Accessories,Wrap yourself in comfort with our cozy shawl. ...,3,"Fall, Winter",22.00
4,CHN0987,Chunky Knit Beanie,Accessories,Keep your head toasty with our chunky knit bea...,2,"Fall, Winter",22.00
...,...,...,...,...,...,...,...
94,SND7654,Strappy Sandals,Women's Shoes,Step into summer with our strappy sandals. The...,8,"Spring, Summer",27.00
95,CHK8901,Chunky Sneakers,Women's Shoes,Step into trendy style with our chunky sneaker...,1,"Spring, Fall",42.00
96,MLR0123,Mule Loafers,Women's Shoes,Slip into effortless style with our mule loafe...,2,"Spring, Fall",47.00
97,CLG4567,Clog Sandals,Women's Shoes,Step into retro-inspired style with our clog s...,3,"Spring, Summer",48.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In..."
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc..."
5,E006,NaN,"Hey there, I was thinking of ordering a pair o..."
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl..."
7,E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your Versatile..."
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr..."
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901..."


In [335]:
if len(products_df['product_id']) == len(products_df['product_id'].unique()):
    print(f"There are no duplicate products in inventory")
else:
    raise ValueError("duplicate products exist in inventory")

There are no duplicate products in inventory


In [336]:
class Product(object):
    def __init__(self, key: str, name: str, category: str, info: str, stock: int, seasons: str, price: str):
        self.id = key
        self.name = name
        self.category = category
        self.description = info
        self.stock = int(stock)
        self.seasons = seasons
        self.price = float(price)

    def create_order(self):
        if self.stock > 0:
            self.stock -= 1
            return True
        return False

class RequestType(Enum):
    Inquiry = 1
    Order = 2

class Request(object):
    def __init__(self, key: int, product: Union[Product, None], email: str, time: int, type: RequestType):
        self.id = key
        self.product = product
        self.email = email
        self.time = time
        self.type = type
        self.successful = False
        self.result: str = None
        self.responded = False

class Inventory(object):
    request_id = -1
    def __init__(self, products: list[Product]):
        self.products: dict[str, Product] = {p.id: p for p in products}
        self.requests: list[Request] = []

    def _get_request_id(self):
        self.request_id += 1
        return self.request_id

    def _get_product(self, id: str):
        if id in self.products:
            return self.products[id]
        else:
            return None

    def _request(self, product_id: str, email: str):
        product = self._get_product(product_id)
        request = Request(self._get_request_id(), product, email, clock.time(), RequestType.Order)
        return product, request

    def inquire(self, id: str, name: str, email: str):
        product, request = self._request(id, email)
        if product:
            request.successful = True
            request.result = product.description
        else:
            request.result = f"The product '{name}' you are inquiring does not exist!"
        self.requests.append(request)
        return request

    def order(self, id: str, name: str, email: str):
        product, request = self._request(id, email)
        if product:
            if product.create_order():
                request.successful = True
                request.result = f"Order on '{product.name}', id #{product.id}, was successful"
            else:
                request.result = f"'{product.name}', id #{product.id}, is out of stock"
        else:
            request.result = f"The product '{name}', you are ordering does not exist!"
        self.requests.append(request)
        return request

    def get_available_products(self):
        return [(key, p.name) for key, p in self.products.items()]

In [337]:
inventory = Inventory([Product(*tuple(row)) for _, row in products_df.iterrows()])

for e, v in vars(random.choice(list(inventory.products.values()))).items():
    print(f"{e} -> {v} (type={type(v)})")

id -> CHN5432 (type=<class 'str'>)
name -> Chunky Knit Sweater (type=<class 'str'>)
category -> Women's Clothing (type=<class 'str'>)
description -> Stay cozy and stylish with our chunky knit sweater. Crafted from soft, high-quality yarn, this oversized sweater features a trendy, chunky knit pattern and a relaxed fit. Perfect for layering or wearing on its own. (type=<class 'str'>)
stock -> 4 (type=<class 'int'>)
seasons -> Fall, Winter (type=<class 'str'>)
price -> 79.99 (type=<class 'float'>)


In [338]:
MAX_TOKEN_NUM = int(4096 * 0.9)
MAX_TOKEN_NUM

3686

# Task 1. Classify emails

In [339]:
class Email(object):
    def __init__(self, key: str, subject: str, message: str):
        self.id = key
        self.subject = subject if 'nan' not in str(subject) else "<NULL>"
        self.message = message

In [340]:
EMAILS = [Email(*tuple(row)) for _, row in emails_df.iterrows()]
for e, v in vars(random.choice(EMAILS)).items():
    print(f"{e} -> {v}")

id -> E014
subject -> Sleek Wallet Order
message -> Please send me 1 Sleek Wallet. Thanks, Johny


In [341]:
def approximate_tokens(text: str):
    # Finding words
    words = len(text.split())
    # Finding symbols
    symbols = len(re.findall(r'[^\w\s]', text))
    return words + symbols
approximate_tokens("Hello there! How're you doing? My name is <FILL>")

14

In [342]:
# Assuming email has been processed into an instance on class Email
def process_email(email: Email):
    prompt = f"Email ID '{email.id}', ref: {email.subject}, msg: {email.message}"
    tokens = approximate_tokens(prompt)
    return prompt, tokens

def batch_process_emails(emails: list[Email], token_limit: int = None):
    if token_limit is None:
        token_limit = MAX_TOKEN_NUM
    batches = []
    batch = ""
    batch_idx = 0
    token_num = 0
    for i, email in enumerate(emails):
        prompt, tokens = process_email(email)
        if token_num + tokens + 3 > token_limit:
            if len(batch) > 0:
                batches.append(batch)
            print(batch)
            batch = ""
            batch_idx = 0
            token_num = 0
        batch += f"\n{batch_idx} => {prompt}"
        batch_idx += 1
        token_num += (tokens + 3)
        # print(batch)
        if len(batches) == 0 and i == len(emails)-1:
            batches.append(batch)
    return batches
print(batch_process_emails(EMAILS)[0])
display(len(batch_process_emails(EMAILS)))


0 => Email ID 'E001', ref: Leather Wallets, msg: Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you!
1 => Email ID 'E002', ref: Buy Vibrant Tote with noise, msg: Good morning, I'm looking to buy the VBT2345 Vibrant Tote bag. My name is Jessica and I love tote bags, they're so convenient for carrying all my stuff. Last summer I bought this really cute straw tote that I used at the beach. Oh, and a few years ago I got this nylon tote as a free gift with purchase that I still use for groceries.
2 => Email ID 'E003', ref: Need your help, msg: Hello, I need a new bag to carry my laptop and documents for work. My name is David and I'm having a hard time deciding which would be better - the LTH1098 Leather Backpack or the Leather Tote? Does one have more organizational pockets than the other? Any insight would be appreciated!
3 => Email ID 'E004', ref: Buy In

1

In [343]:
print(str(inventory.get_available_products()).replace("'", ""))

[(RSG8901, Retro Sunglasses), (SWL2345, Sleek Wallet), (VSC6789, Versatile Scarf), (CSH1098, Cozy Shawl), (CHN0987, Chunky Knit Beanie), (LTH0976, Leather Bifold Wallet), (FZZ1098, Fuzzy Slippers), (FRP9876, Fringe Poncho), (BKR0123, Bucket Hat), (CBY6789, Corduroy Bucket Hat), (CLF2109, Cable Knit Beanie), (LTG5432, Leather Gloves), (CPN3210, Cashmere Poncho), (SFT1098, Infinity Scarf), (ERM5432, Earmuffs), (VBT2345, Vibrant Tote), (SBP4567, Sleek Backpack), (CCB6789, Chic Crossbody), (CCH6789, Chic Clutch), (CBG9876, Canvas Beach Bag), (LTH5432, Leather Tote), (LTH1098, Leather Backpack), (MSK1098, Metallic Clutch), (LTH2109, Leather Messenger Bag), (QTP5432, Quilted Tote), (FRP6789, Fringe Crossbody), (SDE2345, Saddle Bag), (DLD0123, Delightful Dress), (STT4567, Striped Tee), (CPJ2345, Cozy Pajama Set), (LFL7654, Long Fleece Robe), (PTR9876, Patterned Tie), (CSW6789, Cozy Sweater), (DJN8901, Distressed Jeans), (CPL0123, Casual Polo), (RCS0123, Rugged Cargo Shorts), (TLR5432, Tailore

In [344]:
def get_request_types():
    return [(k, v.value) for k, v in vars(RequestType).items() if "_" not in k]

print(get_request_types())

[('Inquiry', 1), ('Order', 2)]


In [345]:
def classify_emails(emails: list[Email]):
    products = str(inventory.get_available_products()).replace("'", "")
    request_types = str(get_request_types()).replace("'", "")
    template = f"Given the following list of available products and their product_id's, {products}." \
               f"and the enum values of the request types, {request_types}"  \
               f"\nOnly reply (in a string format) with a generated Python list of tuples, format" \
               f"(email_id, product_id, product_name, request_type_value), classifying ALL the following email messages " \
               f"into request types 'Inquiries' and 'Orders',  based on the tone and content of the emails. " \
               f"\nNote that if product or product_id is not available the product_id set SHOULD be NULL only if the message reads like the email user was requesting something on the particular product." \
               f"\nNote that emails with multiple product request MUST be split into multiples tuples for each requests." \
               f"\nDO NOT use an apostrophe to represent the strings" \
               f"\nNote that the tuples MUST be in new lines:"
    responses = []
    for batch in batch_process_emails(emails):
        prompt = template + batch
        # print(prompt)

        client = OpenAI(
            # In order to use provided API key, make sure that models you create point to this custom base URL.
            base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
            # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
            api_key='a0BIj000001iX7PMAU'
        )

        completion = client.chat.completions.create(
          model="gpt-4o",
          messages=[
            {"role": "user", "content": prompt}
          ]
        )

        response = completion.choices[0].message
        print(response)
        responses.append(response)
    return responses

responses = classify_emails(EMAILS)

ChatCompletionMessage(content="[E001, LTH0976, Leather Bifold Wallet, 2]\n[E002, VBT2345, Vibrant Tote, 2]\n[E003, LTH1098, Leather Backpack, 1]\n[E003, NULL, Leather Tote, 1]\n[E004, SFT1098, Infinity Scarf, 2]\n[E005, CSH1098, Cozy Shawl, 1]\n[E006, CBT8901, Chelsea Boots, 1]\n[E007, CLF2109, Cable Knit Beanie, 2]\n[E007, FZZ1098, Fuzzy Slippers, 2]\n[E008, VSC6789, Versatile Scarf, 2]\n[E009, DHN0987, Chunky Knit Beanie, 1]\n[E010, RSG8901, Retro Sunglasses, 2]\n[E011, RSG8901, Retro Sunglasses, 1]\n[E012, NULL, Messenger Bag, 1]\n[E013, SND7654, Slide Sandals, 2]\n[E014, SWL2345, Sleek Wallet, 2]\n[E015, NULL, Men's Bag, 1]\n[E016, NULL, Dress, 1]\n[E016, NULL, Travel Bag, 1]\n[E017, NULL, NULL, 2]\n[E018, RSG8901, Retro Sunglasses, 2]\n[E019, CBT8901, Chelsea Boots, 2]\n[E019, FZZ1098, Fuzzy Slippers, 2]\n[E020, SDE2345, Saddle Bag, 1]\n[E021, NULL, Winter Hats, 1]", refusal=None, role='assistant', function_call=None, tool_calls=None)


In [346]:
print(responses[0].content)

[E001, LTH0976, Leather Bifold Wallet, 2]
[E002, VBT2345, Vibrant Tote, 2]
[E003, LTH1098, Leather Backpack, 1]
[E003, NULL, Leather Tote, 1]
[E004, SFT1098, Infinity Scarf, 2]
[E005, CSH1098, Cozy Shawl, 1]
[E006, CBT8901, Chelsea Boots, 1]
[E007, CLF2109, Cable Knit Beanie, 2]
[E007, FZZ1098, Fuzzy Slippers, 2]
[E008, VSC6789, Versatile Scarf, 2]
[E009, DHN0987, Chunky Knit Beanie, 1]
[E010, RSG8901, Retro Sunglasses, 2]
[E011, RSG8901, Retro Sunglasses, 1]
[E012, NULL, Messenger Bag, 1]
[E013, SND7654, Slide Sandals, 2]
[E014, SWL2345, Sleek Wallet, 2]
[E015, NULL, Men's Bag, 1]
[E016, NULL, Dress, 1]
[E016, NULL, Travel Bag, 1]
[E017, NULL, NULL, 2]
[E018, RSG8901, Retro Sunglasses, 2]
[E019, CBT8901, Chelsea Boots, 2]
[E019, FZZ1098, Fuzzy Slippers, 2]
[E020, SDE2345, Saddle Bag, 1]
[E021, NULL, Winter Hats, 1]


In [347]:
responses[0].content.splitlines()[5].split()

['[E005,', 'CSH1098,', 'Cozy', 'Shawl,', '1]']

In [358]:
for txt in responses[0].content.splitlines()[6].split():
    print(re.findall(r'\b\w+\b', txt)[0])

E006
CBT8901
Chelsea
Boots
1


In [362]:
def filter_requests(response):
    def get_request_type(value: int):
        value = int(value)
        types = {v.value: v for k, v in vars(RequestType).items() if "_" not in k}
        if value in types:
            return types[value]
        else:
            raise ValueError(f"Unknown request type '{value}'")
        pass

    def str_combine(strings: list[str]):
        if len(strings) > 1:
            text = strings[0]
            for string in strings[1:]:
                text += f" {string}"
        elif len(strings) == 1:
            text = strings[0]
        else:
            raise ValueError(f"Cannot combine empty list of strings")
        return text

    # Filter the tuples from the string
    validation_pattern_1 = r'.*\(.*E.*\).*'
    validation_pattern_2 = r'.*\[.*E.*\].*'
    requests: list[str] = [line for line in response.content.splitlines() if bool(re.match(validation_pattern_1, line)) or bool(re.match(validation_pattern_2, line))]
    # Filter the elements of the tuple
    filter_pattern = r'\b\w+\b'
    requests: list[list[str]] = [tuple([re.findall(filter_pattern, item)[0] for item in line.split()]) for line in requests]
    # Convert string tuple to an actual tuple
    requests: list[tuple] = [(req[0], req[1], str_combine(req[2:len(req)-1]), get_request_type(req[-1])) for req in requests]
    # Filter the faulty tuples
    requests = [req for req in requests if 'NULL' not in req[2]]
    # Filter request types
    inquiries = [req for req in requests if req[3] == RequestType.Inquiry]
    orders = [req for req in requests if req[3] == RequestType.Order]
    return inquiries, orders

In [363]:
INQUIRIES, ORDERS = [], []
for batch in responses:
    inq, odr = filter_requests(responses[0])
    INQUIRIES += inq
    ORDERS += odr

In [364]:
for r_type in filter_requests(responses[0]):
    print(f"-------------------")
    for t in r_type:
        print(t)
    print(f"-------------------")

-------------------
('E003', 'LTH1098', 'Leather Backpack', <RequestType.Inquiry: 1>)
('E003', 'NULL', 'Leather Tote', <RequestType.Inquiry: 1>)
('E005', 'CSH1098', 'Cozy Shawl', <RequestType.Inquiry: 1>)
('E006', 'CBT8901', 'Chelsea Boots', <RequestType.Inquiry: 1>)
('E009', 'DHN0987', 'Chunky Knit Beanie', <RequestType.Inquiry: 1>)
('E011', 'RSG8901', 'Retro Sunglasses', <RequestType.Inquiry: 1>)
('E012', 'NULL', 'Messenger Bag', <RequestType.Inquiry: 1>)
('E015', 'NULL', 'Men Bag', <RequestType.Inquiry: 1>)
('E016', 'NULL', 'Dress', <RequestType.Inquiry: 1>)
('E016', 'NULL', 'Travel Bag', <RequestType.Inquiry: 1>)
('E020', 'SDE2345', 'Saddle Bag', <RequestType.Inquiry: 1>)
('E021', 'NULL', 'Winter Hats', <RequestType.Inquiry: 1>)
-------------------
-------------------
('E001', 'LTH0976', 'Leather Bifold Wallet', <RequestType.Order: 2>)
('E002', 'VBT2345', 'Vibrant Tote', <RequestType.Order: 2>)
('E004', 'SFT1098', 'Infinity Scarf', <RequestType.Order: 2>)
('E007', 'CLF2109', 'Cable

# Task 2. Process order requests

In [365]:
for order in ORDERS:
    email_id, product_id, product, product_type = order
    inventory.order(product_id, product, email_id)

# Task 3. Handle product inquiry

In [366]:
for inquiry in ORDERS:
    email_id, product_id, product, product_type = inquiry
    inventory.inquire(product_id, product, email_id)

In [378]:
def reply_email(request: Request):
    # Pseudo reply to email
    if not request.responded:
        print(f"For email {request.email} ~ {request.result}")
        request.responded = True


def batch_reply_email(inventory: Inventory):
    for request in inventory.requests:
        reply_email(request)

In [384]:
batch_reply_email(inventory)

# RESET responses
for request in inventory.requests:
    request.responded = False

For email E001 ~ Order on 'Leather Bifold Wallet', id #LTH0976, was successful
For email E002 ~ Order on 'Vibrant Tote', id #VBT2345, was successful
For email E004 ~ Order on 'Infinity Scarf', id #SFT1098, was successful
For email E007 ~ Order on 'Cable Knit Beanie', id #CLF2109, was successful
For email E007 ~ Order on 'Fuzzy Slippers', id #FZZ1098, was successful
For email E008 ~ Order on 'Versatile Scarf', id #VSC6789, was successful
For email E010 ~ Order on 'Retro Sunglasses', id #RSG8901, was successful
For email E013 ~ Order on 'Strappy Sandals', id #SND7654, was successful
For email E014 ~ Order on 'Sleek Wallet', id #SWL2345, was successful
For email E018 ~ 'Retro Sunglasses', id #RSG8901, is out of stock
For email E019 ~ Order on 'Chelsea Boots', id #CBT8901, was successful
For email E019 ~ Order on 'Fuzzy Slippers', id #FZZ1098, was successful
For email E001 ~ Upgrade your everyday carry with our leather bifold wallet. Crafted from premium, full-grain leather, this sleek wal